In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Code Functions below

staging area for mathmatical functions + dataframe manipulation

In [ ]:
def pull_underlying(ticker):
    
    # pull 3-months worth of data to calculate the standard deviation and +/- to the most recent close
    asset = yf.Ticker(ticker)
    price_rn = asset.history(period='1d')
    recent_close = price_rn["Close"].iloc[-1]

    hist_pricing = pd.DataFrame(asset.history(period="12mo"))
    stdev_close = hist_pricing['Close'].std()
    avg_price = hist_pricing['Close'].mean()
    info = {"SYMBOL": ticker, "PRICE": recent_close, "-STDEV": recent_close - stdev_close, "+STDEV": recent_close + stdev_close, "STDEV" : stdev_close, "MU": avg_price}
   
    return info

In [ ]:
def get_option_data(days : int) -> pd.DataFrame:

    # Get the options data for SPY
    options = sec.options

    # Convert the options data to a pandas DataFrame
    options_df = pd.DataFrame(options, columns=['expiration'])
    options_df['expiration'] = pd.to_datetime(options_df['expiration'], format = '%Y-%m-%d')
    options_df['exp_date'] = options_df['expiration'].dt.strftime('%Y-%m-%d')

    # Filter the options data to include only expiries within the next 30 days
    thirty_days_from_now = datetime.now() + timedelta(days=days)
    options_df = options_df[options_df['expiration'].dt.date <= thirty_days_from_now.date()]

    opt_list = []

    for exp in options_df['exp_date']:
        opt = sec.option_chain(date = exp)
        calls = opt.calls
        calls = calls.assign(expiry=exp)
        calls = calls.assign(optType = 'call')
        puts = opt.puts
        puts = puts.assign(expiry=exp)
        puts = puts.assign(optType = 'put')
        opt_list.append(calls)
        opt_list.append(puts)
        opt_chains = pd.concat(opt_list)
    #opt_chains = pd.DataFrame(opt_list, columns=['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask', 'change', 'percentChange', 'volume', 'openInterest', 'impliedVolatility', 'inTheMoney', 'contractSize', 'currency', 'expiry'])
    # Return the filtered options data
    return opt_chains

In [ ]:
def identify_strikes(ticker, days : int):
    mrkt_price = pull_underlying(ticker)
    chains = get_option_data(days)

   
    oi_mean = chains.openInterest.mean()
    oi_stdev = chains.openInterest.std()
    oi_plus1 = oi_mean + oi_stdev
    oi_minus1 = oi_mean - oi_stdev
    
    adj_chains = chains[(chains['strike'] > mrkt_price["-STDEV"]) & (chains['strike'] < mrkt_price["+STDEV"]) & (chains['openInterest'] > oi_minus1)]
    
    return adj_chains

In [ ]:
# fit model and determine optimial params + optimize for pricing discrepancies
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar   
from scipy.optimize import minimize

N = norm.cdf

def merton_jump_call(S, K, T, r, sigma, m , v, lam):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    lambda_t = lam*T
    mu = m - 0.5*v**2
    J = lambda_t*(np.exp(mu + 0.5*v**2)-1)
    N_d1 = N(d1)
    N_d2 = N(d2)
    N_d1_J = N(d1-v*np.sqrt(T))
    call_price = S*np.exp(-J)*N_d1 - K*np.exp(-r*T)*N_d2
    return call_price

def merton_jump_put(S, K, T, r, sigma, m , v, lam):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    lambda_t = lam*T
    mu = m - 0.5*v**2
    J = lambda_t*(np.exp(mu + 0.5*v**2)-1)
    N_d1 = N(d1)
    N_d2 = N(d2)
    N_d1_J = N(d1-v*np.sqrt(T))
    put_price = K*np.exp(-r*T)*(1-N_d2) - S*np.exp(-J)*(1-N_d1_J)
    return put_price


In [ ]:
# function to optimize mu and lambda
def optimal_params(x, mkt_prices, strikes, optType):
    candidate_prices = [merton_jump_call(S, K, T, r, sigma, x[0], iv, x[1]) if optType=="call" else merton_jump_put(S, K, T, r, sigma, x[0], iv, x[1]) for S, K, T, r, sigma, iv in zip(df["S"], strikes, df["t"], df["r"], df["sigma"], df["impliedVolatility"])]
    return np.sum((mkt_prices - candidate_prices) ** 2)

In [ ]:
# triggers pricing across all rows in data frame

### needs to be optimized - maybe with dictionary calling instead of row iteration

def price_by_optType(row):
    if row['optType'] == 'call':
        return merton_jump_call(row['S'], row['strike'], row['t'], row['r'], row['sigma'], row['mu'], row['impliedVolatility'], row['lambda'])
    else:
        return merton_jump_put(row['S'], row['strike'], row['t'], row['r'], row['sigma'], row['mu'], row['impliedVolatility'], row['lambda'])



In [ ]:
# Create a function that runs the minimize function and returns the output
def minimize_helper(row, prices, strikes):
    return minimize(optimal_params, method='SLSQP', x0=x0, args=(prices, strikes, row["optType"]), bounds=bounds, tol=1e-20, options={"maxiter": 1000}).x

In [ ]:
# visualizes the prices of the option instead of underlying security

def merton_jump_paths_with_options(df, S, r, sigma, m, lam, v, steps, Npaths):
    size=(steps,Npaths)
    dt = T/steps 
    poi_rv = np.multiply(np.random.poisson( lam*dt, size=size), np.random.normal(m,v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r -  sigma**2/2 -lam*(m  + v**2*0.5))*dt + sigma*np.sqrt(dt) * np.random.normal(size=size)), axis=0)
    stock_prices = np.exp(geo+poi_rv)*S
    
    df["price"] = df.apply(lambda row: price_by_optType(row), axis=1)
    return df.price.tolist()

# this is for the underlying asset
def merton_jump_paths(S, T, r, sigma,  lam, m, v, steps, Npaths):
    size=(steps,Npaths)
    dt = T/steps 
    poi_rv = np.multiply(np.random.poisson( lam*dt, size=size),
                         np.random.normal(m,v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r -  sigma**2/2 -lam*(m  + v**2*0.5))*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=size)), axis=0)
    
    return np.exp(geo+poi_rv)*S

# Insert Ticker Symbol and Risk-Free Rate as a decimal below to begin analysis

- Risk Free Rate is from: https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates&field_tdr_date_value_month=202301
    - Based on either 4-8 week bank discount

In [77]:
t_str = "abnb"

In [78]:
# HERE - fixed variables that will require manipulation
ticker = (f"{t_str}").upper()
sec = yf.Ticker(ticker)

# adjust rate based on macro outlook
rfr = 0.05
# duration of options (influences rfr as well)
day = 30
#days out of calendar year
min_maturity = 14/365

## Call functions above to model prices

- pulls data
- aggregates greeks
- performs calculation
- drops uncessary rows
- sorts by p&L

In [79]:
# call functions to pull data based on provided inputs
opt_df = identify_strikes(ticker, days=day)
asset_info = pull_underlying(ticker)

In [80]:
# dataframe manipulation - greek aggregation for S, T, r, sigma + midpoint price of option

df = opt_df[['contractSymbol', 'expiry', 'strike', 'lastPrice', 'bid', 'ask', 'change', 'percentChange', 'impliedVolatility', 'optType', 'openInterest']]
### start fixed column variables
df['S'] = asset_info['PRICE']
df = df.assign(t = (pd.to_datetime(df['expiry']) - pd.Timestamp.today()).dt.days)
df['t'] = np.where(df['t'] < 0, 1, df['t'])
df = df.assign(r = rfr)

# need to divide by a full calendar year that's scalar with the expiration
df['t'] = df['t']/365
df = df.assign(sigma = (asset_info['STDEV'] / asset_info['MU']))
### END FIXED COLUMNS

# keep midpoint here for table readability later
df = df.assign(midpoint = ((df.bid + df.ask)/2))
df = df[df['t'] > min_maturity]

df.shape

/var/folders/_x/rmbn1d5n4cx4ht3tt724nzrc0000gn/T/ipykernel_59830/3399871843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['S'] = asset_info['PRICE']


(132, 16)

**parallelized the process across all cores - dictionary referencing didn't create any incremental performance gains**

In [81]:
# parallelize job to make run faster
from joblib import Parallel, delayed
# order inherently for (sigma (annual volatilty), mu (mean jump size), v (std dev of jump size - impliedVolatility), lambda(number of jumps per year intensity))

# jump mean and lambda (intensity)
x0 = [0.1, 1]  # initial guess for algorithm
bounds = ((0.01, 2), (0, 5))  # bounds for parameters
strikes = df.strike.values
prices = df.midpoint.values

# Use joblib to run the minimize_helper function in parallel
df["params"] = Parallel(n_jobs=-1)(delayed(minimize_helper)(row, prices, strikes) for _, row in df.iterrows())

# increase verbosity when complete so I can tell
print(f"Parameter Optimization for - {ticker}: COMPLETE")

Parameter Optimization for - ABNB: COMPLETE


In [82]:
# split mu and lamda from list to call pricing function
## to lazy to call the list w/in the series

df[['mu', 'lambda']] = df.params.apply(pd.Series)

In [83]:
df['price'] = df.apply(price_by_optType, axis=1)

df['price_diff'] = df['price'] - df['midpoint']
df['p&L'] = (df['price'] - df['midpoint']) * 100
df['percentage_growth'] = (df['price_diff']/df['midpoint']) * 100
df = df[df['midpoint'] > 0]

In [84]:
udf = df.drop(columns=['S', 't', 'r', 'sigma', 'params', 'mu', 'lambda'])

In [85]:
udf = udf[['contractSymbol', 'strike', 'expiry','optType', 'price', 'midpoint','lastPrice', 
       'p&L', 'price_diff', 'bid', 'ask', 'impliedVolatility', 
       'change', 'percentChange',  'openInterest']]

In [86]:
sorted_pricediff = udf.sort_values(by=['openInterest','price_diff', 'impliedVolatility'], ascending=False)
sorted_pricediff = sorted_pricediff[sorted_pricediff['price_diff'] > 0]
sorted_pricediff.head()

,contractSymbol,strike,expiry,optType,price,midpoint,lastPrice,p&L,price_diff,bid,ask,impliedVolatility,change,percentChange,openInterest


In [87]:
best_pL_info = df.loc[df['contractSymbol'] == sorted_pricediff.contractSymbol.values[0]]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
#set by user
### UPDATE THESE ONLY ###
npaths = 1
# set above when entering ticker
steps = day

#set by model
S = asset_info['PRICE']
T = best_pL_info['t'].values[0]
m = best_pL_info['mu'].values[0]
lam = best_pL_info['lambda'].values[0]
v = best_pL_info['impliedVolatility'].values[0]
sigma = best_pL_info['sigma'].values[0]
strike = best_pL_info['strike'].values[0]
optType = best_pL_info['optType'].values[0]

In [ ]:
ul = merton_jump_paths(S=S, T=T, r=rfr, sigma=sigma, lam=lam, m=m, v=v, steps=steps, Npaths=npaths)

In [ ]:
x = np.arange(len(ul))
y = ul.reshape(-1)
time2exp = (pd.to_datetime(best_pL_info['expiry'].values[0]) - pd.Timestamp.today()).days
y_coordinate = round(y[time2exp], 2)
diff = (y - S) / S * 100

sns.lineplot(x=x, y=y)
sns.set_theme('talk')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.title('Jump Diffusion Process')

plt.axvline(time2exp, color='black', linestyle='dashed', linewidth=2,label="day2expiry")
plt.axhline(S, color='red', linestyle='dashed', linewidth=2,label="current price")
plt.axhline(strike, color='green', linestyle='dashed', linewidth=2,label="strike")
plt.axhline(asset_info['+STDEV'], color='black', linestyle='dashed', linewidth=2,label="+1")
plt.axhline(asset_info['-STDEV'], color='black', linestyle='dashed', linewidth=2,label="-1")

plt.annotate(f'${round(y[time2exp], 2)}', xy=(time2exp, y[time2exp]), xytext=(time2exp-1, y[time2exp]-1),
             arrowprops=dict(facecolor='black', shrink=0.05, linewidth=0))
plt.fill_between(x, y_coordinate, S, color='gray', alpha=0.5)

diff = (y[time2exp]-S)
perc_diff = (diff/S)*100
plt.annotate(f"Projected Price Drop: ${diff:.2f} | {perc_diff:.2f}%", xy=(time2exp, y[time2exp]), xytext=(time2exp+1, y[time2exp]+1),
             fontweight='bold', color='green',arrowprops=dict(arrowstyle='-', color='green', linewidth=0))

In [ ]:
time2exp = (pd.to_datetime(best_pL_info['expiry'].values[0]) - pd.Timestamp.today()).days

j = merton_jump_paths_with_options(df, S=S, r=rfr, sigma=sigma, m=m, lam=lam, v=v, steps=time2exp, Npaths=npaths)
x = np.arange(len(j))
y = np.maximum(j, 0)
#y_coordinate = round(y[time2exp], 2)
diff = (y - S) / S * 100

plt.xlim(0, time2exp)
sns.lineplot(x=x, y=y)
sns.set_theme('talk')
plt.xlabel('Days')
plt.ylabel('Option Price')
plt.title('Jump Diffusion Price')

plt.axhline(best_pL_info['bid'].values[0], color='green', linestyle='dashed')
plt.axhline(best_pL_info['ask'].values[0], color='red', linestyle='dashed')
plt.axhline(best_pL_info['lastPrice'].values[0], color='black', linestyle='dashed')



In [ ]:
sorted_pricediff